# Import packages

In [1]:
import pandas as pd
from teradataml import *

from modules.db import TeradataDatabase
from utils import clean_text, load_embedding_model
from constants import TEST_DATA_PATH, CLASS_EMBEDDINGS_PATH, PRODUCT_TEST_EMBEDDINGS_PATH
from modules.models import SentenceEmbeddingModel, SentenceEmbeddingConfig

Logger initialized. All logs will be saved to: c:\Users\ss255385\OneDrive - Teradata Corporation\Desktop\AMuRD-Iteration-7\src\logs\borai_20250817_165215.log


## Connect to database

In [2]:
td_db = TeradataDatabase()
td_db.connect()

## Load data to database

In [ ]:
df = pd.read_csv(TEST_DATA_PATH)
df.head()

In [ ]:
df = df[["Item_Name", "Brand"]]
df.rename(columns={"Item_Name": "product_name", "Brand": "brand_name"}, inplace=True)

In [ ]:
df.drop_duplicates(subset=["product_name"], inplace=True)
df.dropna(subset=["product_name"], inplace=True)
df

In [ ]:
df["brand_name"] = df["brand_name"].astype(str)

In [ ]:
tdf = DataFrame(df)

In [ ]:
tdf = tdf.assign(product_id=tdf.index_label)
tdf = tdf.drop(columns=["index_label"])
tdf

In [ ]:
tdf = tdf.select(["product_id", "product_name", "brand_name"])
tdf

In [ ]:
copy_to_sql(tdf, "products", "amurd", if_exists="replace")

## Load table from database

In [12]:
",\n ".join([f"dim_{i} FLOAT" for i in range(1024)])

'dim_0 FLOAT,\n dim_1 FLOAT,\n dim_2 FLOAT,\n dim_3 FLOAT,\n dim_4 FLOAT,\n dim_5 FLOAT,\n dim_6 FLOAT,\n dim_7 FLOAT,\n dim_8 FLOAT,\n dim_9 FLOAT,\n dim_10 FLOAT,\n dim_11 FLOAT,\n dim_12 FLOAT,\n dim_13 FLOAT,\n dim_14 FLOAT,\n dim_15 FLOAT,\n dim_16 FLOAT,\n dim_17 FLOAT,\n dim_18 FLOAT,\n dim_19 FLOAT,\n dim_20 FLOAT,\n dim_21 FLOAT,\n dim_22 FLOAT,\n dim_23 FLOAT,\n dim_24 FLOAT,\n dim_25 FLOAT,\n dim_26 FLOAT,\n dim_27 FLOAT,\n dim_28 FLOAT,\n dim_29 FLOAT,\n dim_30 FLOAT,\n dim_31 FLOAT,\n dim_32 FLOAT,\n dim_33 FLOAT,\n dim_34 FLOAT,\n dim_35 FLOAT,\n dim_36 FLOAT,\n dim_37 FLOAT,\n dim_38 FLOAT,\n dim_39 FLOAT,\n dim_40 FLOAT,\n dim_41 FLOAT,\n dim_42 FLOAT,\n dim_43 FLOAT,\n dim_44 FLOAT,\n dim_45 FLOAT,\n dim_46 FLOAT,\n dim_47 FLOAT,\n dim_48 FLOAT,\n dim_49 FLOAT,\n dim_50 FLOAT,\n dim_51 FLOAT,\n dim_52 FLOAT,\n dim_53 FLOAT,\n dim_54 FLOAT,\n dim_55 FLOAT,\n dim_56 FLOAT,\n dim_57 FLOAT,\n dim_58 FLOAT,\n dim_59 FLOAT,\n dim_60 FLOAT,\n dim_61 FLOAT,\n dim_62 FLOAT,\n d

In [19]:
query = f"""
CREATE MULTISET TABLE amurd.p_embeddings (
    product_id INTEGER NOT NULL,
    {",\n ".join([f"dim_{i} FLOAT" for i in range(1024)])}
) PRIMARY INDEX (product_id);
"""

In [21]:
tdf = td_db.execute_query(query)

Exception: Query execution failed: [Version 20.0.0.34] [Session 1573] [Teradata Database] [Error 3803] Table 'p_embeddings' already exists.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:83
 at gosqldriver/teradatasql.(*teradataConnection).formatDatabaseError ErrorUtil.go:190
 at gosqldriver/teradatasql.(*teradataConnection).makeChainedDatabaseError ErrorUtil.go:206
 at gosqldriver/teradatasql.(*teradataConnection).processErrorParcel TeradataConnection.go:340
 at gosqldriver/teradatasql.(*TeradataRows).processResponseBundle TeradataRows.go:2597
 at gosqldriver/teradatasql.(*TeradataRows).executeSQLRequest TeradataRows.go:1073
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:792
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:823
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1786
 at database/sql.withLock

In [3]:
df = pd.read_csv(PRODUCT_TEST_EMBEDDINGS_PATH)

In [5]:
df['embeddings'].iloc[0]

'[0.00963592529296875, 0.02484130859375, 0.0007691383361816406, -0.05615234375, 0.0118560791015625, -0.028076171875, -0.007808685302734375, 0.01800537109375, 0.0249176025390625, -0.0257720947265625, 0.0245361328125, 0.00820159912109375, -0.03216552734375, -0.0125274658203125, -0.0224761962890625, -0.0284271240234375, -0.033660888671875, 0.0323486328125, -0.006015777587890625, -0.0100250244140625, 0.03485107421875, -0.00897216796875, -0.040252685546875, -0.039215087890625, -0.017791748046875, 0.00446319580078125, -0.033355712890625, -0.03192138671875, -0.049072265625, -0.004787445068359375, -0.01050567626953125, 0.0243377685546875, -0.0310211181640625, -0.06396484375, -0.028228759765625, 0.053955078125, 0.0374755859375, 0.014007568359375, -0.037628173828125, 0.035003662109375, -0.00463104248046875, 0.0278778076171875, 0.01702880859375, -0.038330078125, -0.0220794677734375, 0.032440185546875, 0.021728515625, 0.0052490234375, 1.0251998901367188e-05, 0.047515869140625, 0.036163330078125, 0

In [20]:
import json
# for row in df.iterrows():
#     id = row[0]
#     embeddings = row[1]
#     print(id, embeddings)
#     break
l = json.loads(df.iloc[0, 1])
l = [str(embed) for embed in l]
queries = ""
for i in range(len(df)):
    id = df.iloc[i, 0]
    embeddings = json.loads(df.iloc[i, 1])
    embeddings = [str(embed) for embed in embeddings]
    queries += f"""
INSERT INTO amurd.p_embeddings VALUES ({id}, {embeddings});\n
"""

In [22]:
print(queries)

In [ ]:
embeddings = pd.DataFrame(df['embeddings'].tolist(),
                                   columns=[f"dim_{i}" for i in range(1024)])
df_expanded = pd.concat([df[['id']], embeddings], axis=1)

In [ ]:
query = f"""
CREATE MULTISET TABLE amurd.c_embeddings (
    class_id INTEGER NOT NULL,
    {",\n ".join([f"dim_{i} FLOAT" for i in range(1024)])}
) PRIMARY INDEX (class_id);
"""

In [ ]:
class_embedding_df = pd.read_csv(CLASS_EMBEDDINGS_PATH)

In [ ]:
class_embedding_df = pd.read_csv(CLASS_EMBEDDINGS_PATH)
class_id = class_embedding_df["id"]
embedding = class_embedding_df["embedding"]

In [ ]:
df_new = pd.DataFrame({
    "product_id": class_id,
    "embedding": embedding
})

tdf_new = DataFrame(df_new, types={"embedding": VECTOR})
# copy_to_sql(df_new, table_name="product_embeddings", if_exists="replace", types={"embedding": BLOB(4096)})

## Disconnect

In [ ]:
td_db.disconnect()